# 有馬記念に勝ちたい
ここら辺を参考に予測器を作って有馬記念に勝ちたい

- https://qiita.com/yossymura/items/334a8f3ef85bff081913
- https://qiita.com/ishizakiiii/items/3b894b6e987fdf87093e
- https://qiita.com/kami634/items/55e49dad76396d808bf5
- https://qiita.com/teco_ito/items/171865d1d6227a5e85e8

中山競馬場での予測器を作る

## データの収集
中山競馬場の各レースについて、出走馬の当日のデータ、過去のレースのデータをhttps://db.netkeiba.com/　からとってくる

[レースごとのページ](https://db.netkeiba.com/race/201908050311/)、[馬ごとのページ](https://db.netkeiba.com/horse/2017105327/)があって、馬ごとのページのみ見れば十分なので、中山競馬場のレースに出走した馬のurlを収集してそこからデータを作る

しかし、中山競馬場に出走した馬という検索ができないので、中山競馬場のレース->各レースページ->レースに出走した馬のページ  とアクセスする

In [71]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import sqlite3
from contextlib import closing
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm_notebook as tqdm
import time
import re
import datetime
from bs4 import BeautifulSoup
import requests

options = Options()
options.add_argument('--headless')    # ヘッドレスモードに
driver = webdriver.Chrome(chrome_options=options) 
wait = WebDriverWait(driver,10)

/Users/ht/.pyenv/versions/miniconda3-latest/envs/cert/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


In [41]:
URL = "https://db.netkeiba.com/?pid=race_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

### 中山競馬場のレースのurlを収集

In [3]:
# 中山を選択
driver.find_element_by_id("check_Jyo_06").click()
# 新馬、未勝利、未出走以外の芝、ダートだけ選択
driver.find_element_by_id("check_track_1").click()
driver.find_element_by_id("check_track_2").click()
driver.find_element_by_id("check_grade_1").click()
driver.find_element_by_id("check_grade_2").click()
driver.find_element_by_id("check_grade_3").click()
driver.find_element_by_id("check_grade_4").click()
driver.find_element_by_id("check_grade_5").click()
driver.find_element_by_id("check_grade_6").click()
driver.find_element_by_id("check_grade_7").click()

# 表示件数を選択(20,50,100の中から最大の100へ)
list_element = driver.find_element_by_name('list')
list_select = Select(list_element)
list_select.select_by_value("100")
# フォームを送信
frm = driver.find_element_by_css_selector("#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

In [4]:
# url を収集
df = pd.DataFrame(columns='race_url year month date'.split(' '))

while True:
    time.sleep(0.5)
    wait.until(EC.presence_of_all_elements_located)
    all_rows = driver.find_element_by_class_name('race_table_01').find_elements_by_tag_name("tr")
    for row in range(1, len(all_rows)):
        tds = all_rows[row].find_elements_by_tag_name("td")
        year = tds[0].find_element_by_tag_name("a").text
        race_href = tds[4].find_element_by_tag_name("a").get_attribute("href")
        tmp_se = pd.Series([race_href]+[int(i) for i in year.split('/')], index=df.columns)
        df = df.append(tmp_se, ignore_index=True)
    try:
        target = driver.find_elements_by_link_text("次")[0]
        driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
    except IndexError:
        break
df.to_csv('./data/races.csv')
df

,race_url,year,month,date
0,https://db.netkeiba.com/race/201906050607/,2019,12,15
1,https://db.netkeiba.com/race/201906050608/,2019,12,15
2,https://db.netkeiba.com/race/201906050609/,2019,12,15
3,https://db.netkeiba.com/race/201906050610/,2019,12,15
4,https://db.netkeiba.com/race/201906050611/,2019,12,15
...,...,...,...,...
8130,https://db.netkeiba.com/race/195506040810/,1955,10,16
8131,https://db.netkeiba.com/race/195506040410/,1955,10,2
8132,https://db.netkeiba.com/race/195506010401/,1955,1,8
8133,https://db.netkeiba.com/race/195406060410/,1954,12,12


In [5]:
df[df['year']>=2000]

,race_url,year,month,date
0,https://db.netkeiba.com/race/201906050607/,2019,12,15
1,https://db.netkeiba.com/race/201906050608/,2019,12,15
2,https://db.netkeiba.com/race/201906050609/,2019,12,15
3,https://db.netkeiba.com/race/201906050610/,2019,12,15
4,https://db.netkeiba.com/race/201906050611/,2019,12,15
...,...,...,...,...
5039,https://db.netkeiba.com/race/200006010211/,2000,1,6
5040,https://db.netkeiba.com/race/200006010107/,2000,1,5
5041,https://db.netkeiba.com/race/200006010108/,2000,1,5
5042,https://db.netkeiba.com/race/200006010109/,2000,1,5


### レースに出走した馬のurlを収集

途中で気づいたけどBeautiful Soup でやった方が速かった

In [20]:
# urls = []
for i, url in enumerate(tqdm(df['race_url'])):
    # 一回止まったので途中から再開
    if i < 5619:
        continue
    driver.get(url)
    time.sleep(1)
    wait.until(EC.presence_of_all_elements_located)
    try:
        all_rows = driver.find_element_by_xpath('//table[@class="race_table_01 nk_tb_common"]').find_elements_by_tag_name("tr")
    except NoSuchElementException:
        print(url)
        continue
    for row in range(1, len(all_rows)):
        tds = all_rows[row].find_elements_by_tag_name("td")
        url_href = tds[3].find_element_by_tag_name("a").get_attribute("href")
        urls.append(url_href)
urls
with open('./data/horse_urls.pkl', 'wb') as f:
    pickle.dump(list(set(urls)), f)




  0%|          | 0/8135 [00:00<?, ?it/s]


 69%|██████▉   | 5620/8135 [00:03<00:01, 1743.19it/s]


 69%|██████▉   | 5620/8135 [00:22<00:01, 1743.19it/s]


 69%|██████▉   | 5629/8135 [00:23<28:27,  1.47it/s]  


 69%|██████▉   | 5630/8135 [00:25<46:49,  1.12s/it]


 69%|██████▉   | 5631/8135 [00:28<1:08:01,  1.63s/it]


 69%|██████▉   | 5632/8135 [00:31<1:20:20,  1.93s/it]


 69%|██████▉   | 5633/8135 [00:33<1:24:19,  2.02s/it]


 69%|██████▉   | 5634/8135 [00:35<1:20:09,  1.92s/it]


 69%|██████▉   | 5635/8135 [00:37<1:20:41,  1.94s/it]


 69%|██████▉   | 5636/8135 [00:39<1:25:45,  2.06s/it]


 69%|██████▉   | 5637/8135 [00:41<1:25:15,  2.05s/it]


 69%|██████▉   | 5638/8135 [00:43<1:27:01,  2.09s/it]


 69%|██████▉   | 5639/8135 [00:45<1:27:11,  2.10s/it]


 69%|██████▉   | 5640/8135 [00:48<1:32:58,  2.24s/it]


 69%|██████▉   | 5641/8135 [00:50<1:30:44,  2.18s/it]


 69%|██████▉   | 5642/8135 [00:53<1:38:12,  2.36s/it]


 69%|██████▉   | 5643/8135 [00:54<1:29:58,  2.17s/it]


 69%

 71%|███████   | 5752/8135 [05:04<1:26:43,  2.18s/it]


 71%|███████   | 5753/8135 [05:06<1:32:57,  2.34s/it]


 71%|███████   | 5754/8135 [05:08<1:26:15,  2.17s/it]


 71%|███████   | 5755/8135 [05:10<1:23:32,  2.11s/it]


 71%|███████   | 5756/8135 [05:12<1:19:48,  2.01s/it]


 71%|███████   | 5757/8135 [05:14<1:19:22,  2.00s/it]


 71%|███████   | 5758/8135 [05:16<1:17:10,  1.95s/it]


 71%|███████   | 5759/8135 [05:18<1:17:44,  1.96s/it]


 71%|███████   | 5760/8135 [05:20<1:15:47,  1.91s/it]


 71%|███████   | 5761/8135 [05:21<1:14:30,  1.88s/it]


 71%|███████   | 5762/8135 [05:23<1:14:38,  1.89s/it]


 71%|███████   | 5763/8135 [05:25<1:11:47,  1.82s/it]


 71%|███████   | 5764/8135 [05:27<1:11:35,  1.81s/it]


 71%|███████   | 5765/8135 [05:29<1:12:30,  1.84s/it]


 71%|███████   | 5766/8135 [05:31<1:14:18,  1.88s/it]


 71%|███████   | 5767/8135 [05:33<1:16:07,  1.93s/it]


 71%|███████   | 5768/8135 [05:34<1:14:22,  1.89s/it]


 71%|███████   | 5769/8135 [05:37<1:18:10,  1.98

 72%|███████▏  | 5878/8135 [09:47<1:18:19,  2.08s/it]


 72%|███████▏  | 5879/8135 [09:49<1:13:31,  1.96s/it]


 72%|███████▏  | 5880/8135 [09:50<1:10:01,  1.86s/it]


 72%|███████▏  | 5881/8135 [09:52<1:08:18,  1.82s/it]


 72%|███████▏  | 5882/8135 [09:54<1:08:56,  1.84s/it]


 72%|███████▏  | 5883/8135 [09:56<1:08:57,  1.84s/it]


 72%|███████▏  | 5884/8135 [09:58<1:08:50,  1.83s/it]


 72%|███████▏  | 5885/8135 [09:59<1:07:54,  1.81s/it]


 72%|███████▏  | 5886/8135 [10:01<1:08:58,  1.84s/it]


 72%|███████▏  | 5887/8135 [10:04<1:15:12,  2.01s/it]


 72%|███████▏  | 5888/8135 [10:05<1:12:47,  1.94s/it]


 72%|███████▏  | 5889/8135 [10:08<1:13:58,  1.98s/it]


 72%|███████▏  | 5890/8135 [10:09<1:11:42,  1.92s/it]


 72%|███████▏  | 5891/8135 [10:11<1:14:23,  1.99s/it]


 72%|███████▏  | 5892/8135 [10:13<1:12:26,  1.94s/it]


 72%|███████▏  | 5893/8135 [10:15<1:11:36,  1.92s/it]


 72%|███████▏  | 5894/8135 [10:19<1:29:44,  2.40s/it]


 72%|███████▏  | 5895/8135 [10:21<1:29:55,  2.41

 74%|███████▍  | 6004/8135 [14:12<1:26:24,  2.43s/it]


 74%|███████▍  | 6005/8135 [14:15<1:23:58,  2.37s/it]


 74%|███████▍  | 6006/8135 [14:16<1:18:33,  2.21s/it]


 74%|███████▍  | 6007/8135 [14:19<1:17:56,  2.20s/it]


 74%|███████▍  | 6008/8135 [14:21<1:23:37,  2.36s/it]


 74%|███████▍  | 6009/8135 [14:24<1:23:56,  2.37s/it]


 74%|███████▍  | 6010/8135 [14:26<1:23:45,  2.37s/it]


 74%|███████▍  | 6011/8135 [14:28<1:19:21,  2.24s/it]


 74%|███████▍  | 6012/8135 [14:30<1:16:28,  2.16s/it]


 74%|███████▍  | 6013/8135 [14:32<1:17:05,  2.18s/it]


 74%|███████▍  | 6014/8135 [14:35<1:28:27,  2.50s/it]


 74%|███████▍  | 6015/8135 [14:37<1:21:53,  2.32s/it]


 74%|███████▍  | 6016/8135 [14:39<1:17:17,  2.19s/it]


 74%|███████▍  | 6017/8135 [14:41<1:15:29,  2.14s/it]


 74%|███████▍  | 6018/8135 [14:44<1:18:18,  2.22s/it]


 74%|███████▍  | 6019/8135 [14:46<1:15:16,  2.13s/it]


 74%|███████▍  | 6020/8135 [14:48<1:16:26,  2.17s/it]


 74%|███████▍  | 6021/8135 [14:50<1:16:24,  2.17

 75%|███████▌  | 6130/8135 [19:09<1:09:02,  2.07s/it]


 75%|███████▌  | 6131/8135 [19:11<1:09:01,  2.07s/it]


 75%|███████▌  | 6132/8135 [19:13<1:07:56,  2.04s/it]


 75%|███████▌  | 6133/8135 [19:15<1:09:14,  2.08s/it]


 75%|███████▌  | 6134/8135 [19:17<1:09:25,  2.08s/it]


 75%|███████▌  | 6135/8135 [19:19<1:09:58,  2.10s/it]


 75%|███████▌  | 6136/8135 [19:21<1:11:54,  2.16s/it]


 75%|███████▌  | 6137/8135 [19:23<1:09:30,  2.09s/it]


 75%|███████▌  | 6138/8135 [19:25<1:09:15,  2.08s/it]


 75%|███████▌  | 6139/8135 [19:27<1:05:06,  1.96s/it]


 75%|███████▌  | 6140/8135 [19:29<1:03:56,  1.92s/it]


 75%|███████▌  | 6141/8135 [19:31<1:04:46,  1.95s/it]


 76%|███████▌  | 6142/8135 [19:33<1:08:36,  2.07s/it]


 76%|███████▌  | 6143/8135 [19:35<1:04:45,  1.95s/it]


 76%|███████▌  | 6144/8135 [19:37<1:06:46,  2.01s/it]


 76%|███████▌  | 6145/8135 [19:39<1:04:25,  1.94s/it]


 76%|███████▌  | 6146/8135 [19:41<1:09:50,  2.11s/it]


 76%|███████▌  | 6147/8135 [19:43<1:08:12,  2.06

...

https://db.netkeiba.com/race/196206030508/





 97%|█████████▋| 7895/8135 [1:10:49<06:05,  1.52s/it]

https://db.netkeiba.com/race/196206030510/





 97%|█████████▋| 7896/8135 [1:10:51<06:37,  1.67s/it]


 97%|█████████▋| 7897/8135 [1:10:52<06:31,  1.65s/it]


 97%|█████████▋| 7898/8135 [1:10:54<06:15,  1.58s/it]


 97%|█████████▋| 7899/8135 [1:10:56<06:17,  1.60s/it]


 97%|█████████▋| 7900/8135 [1:10:57<06:08,  1.57s/it]


 97%|█████████▋| 7901/8135 [1:10:59<06:02,  1.55s/it]


 97%|█████████▋| 7902/8135 [1:11:00<06:21,  1.64s/it]


 97%|█████████▋| 7903/8135 [1:11:02<06:02,  1.56s/it]


 97%|█████████▋| 7904/8135 [1:11:03<05:56,  1.54s/it]


 97%|█████████▋| 7905/8135 [1:11:05<05:50,  1.52s/it]


 97%|█████████▋| 7906/8135 [1:11:07<06:16,  1.64s/it]


 97%|█████████▋| 7907/8135 [1:11:08<06:02,  1.59s/it]


 97%|█████████▋| 7908/8135 [1:11:10<05:52,  1.55s/it]


 97%|█████████▋| 7909/8135 [1:11:11<05:46,  1.53s/it]


 97%|█████████▋| 7910/8135 [1:11:13<05:57,  1.59s/it]


 97%|█████████▋| 7911/8135 [1:11:15<06:09,  1.65s/it]


 97%|█████████▋| 7912/8135 [1:11:16<05:57,  1.60s/it]


 97%|█████████▋| 7913/8135 [1:11:18<05:46,  1

 99%|█████████▊| 8022/8135 [1:14:11<02:44,  1.46s/it]


 99%|█████████▊| 8023/8135 [1:14:13<02:43,  1.46s/it]


 99%|█████████▊| 8024/8135 [1:14:14<02:42,  1.47s/it]


 99%|█████████▊| 8025/8135 [1:14:16<02:43,  1.48s/it]


 99%|█████████▊| 8026/8135 [1:14:17<02:47,  1.53s/it]


 99%|█████████▊| 8027/8135 [1:14:19<02:41,  1.49s/it]


 99%|█████████▊| 8028/8135 [1:14:20<02:39,  1.49s/it]


 99%|█████████▊| 8029/8135 [1:14:22<02:37,  1.49s/it]


 99%|█████████▊| 8030/8135 [1:14:23<02:39,  1.51s/it]


 99%|█████████▊| 8031/8135 [1:14:25<02:37,  1.51s/it]


 99%|█████████▊| 8032/8135 [1:14:27<02:43,  1.59s/it]


 99%|█████████▊| 8033/8135 [1:14:28<02:39,  1.57s/it]


 99%|█████████▉| 8034/8135 [1:14:30<02:34,  1.53s/it]


 99%|█████████▉| 8035/8135 [1:14:31<02:43,  1.63s/it]


 99%|█████████▉| 8036/8135 [1:14:33<02:37,  1.59s/it]


 99%|█████████▉| 8037/8135 [1:14:34<02:33,  1.57s/it]


 99%|█████████▉| 8038/8135 [1:14:36<02:30,  1.55s/it]


 99%|█████████▉| 8039/8135 [1:14:37<02:26,  1.53

### (中山競馬場のレースに出走した)馬の出走履歴の表をデータベースに突っ込む

#### 表

|idx|attr||
|-----|-----|-----|
|0|日付|2019/12/15|
|1|会場r|5中山6|
|2|天気|晴
|3|R|7
|4|レース名(href=race id)|href|
|5|映像(null)||
|6|頭数|14|
|7|枠番|5|
|8|馬番|7|
|9|odds|9.1|
|10|人気|4|
|11|着順|1|
|12|騎手|武豊|
|13|斤量|57|
|14|距離|ダ2500|
|15|馬場|良|
|16|馬場指数(null)||
|17|タイム|2:42.8|
|18|着差|2.1|
|19|タイム指数(null)||
|20|通過(2-2, 2-3-4)||
|21|ペース(float-float)|31.0-38.6|
|22|上り|38.0|
|23|馬体重(実体重(+-体重))|538(-12)|
|24|あとわ使わん|

In [84]:
import re
import datetime

weight_ptn = re.compile(r'([0-9]+)\(([+-]?[0-9]+)\)')
place_ptn = re.compile(r'[0-9]*([^0-9]+)[0-9]*')
last_nums_ptn = re.compile(r'.*/([0-9]+)/')

data_index = 'horse_id R head_count wakuban umaban popularity order burden_weight odds time_gap up weather baba raceid place pace0 pace1 race_type distance weight weight_change date time'.split(' ')

def trynone(func, inp):
    try:
        return func(inp)
    except:
        return None
    
def fixed_length(l, n):
    return l + [None for _ in range(n-len(l))]

def tds2data(tds, horse_id, timer=None):
    intdata = [trynone(int, td.text) for td in np.array(tds)[[3, 6, 7, 8, 10, 11, 13]]]
    floatdata = [trynone(float, td.text) for td in np.array(tds)[[9, 18, 22]]]
    textdata = [td.text for td in np.array(tds)[[2, 15]]]
    raceid = [trynone(int, re.findall(last_nums_ptn, tds[4].find("a").get("href"))[0])]
    place = fixed_length(re.findall(place_ptn, tds[1].text), 1)
    pace = fixed_length([trynone(float, i) for i in tds[21].text.split('-')], 2)
    type_n_dist = [tds[14].text[0], trynone(int, tds[14].text[1:])]
    weight = fixed_length([trynone(int, i) for i in re.findall(weight_ptn, tds[23].text)[0]], 2)
    date = [datetime.datetime.strptime(tds[0].text, '%Y/%m/%d')]
    time = [datetime.datetime.strptime(tds[17].text, '%M:%S.%U')]
    data = [trynone(int, re.findall(last_nums_ptn, horse_id)[0])] + intdata + floatdata + textdata + raceid + place + pace + type_n_dist + weight + date + time
    return tuple(data)

In [ ]:
import sqlite3
from contextlib import closing
from tqdm import tqdm_notebook as tqdm
import time

class Timer:
    def __init__(self):
        t = time.time()
        self.start = t
        self.rap_start = t
    
    def time(self, description=None):
        t = time.time()
        print(f'{description}\nabs_time: {t-self.start}\nrap_time: {t-self.rap_start}')
        self.rap_start = t

dbpath = './data/horse.db'
if True:
    with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
        c = conn.cursor()
        c.execute('drop table if exists race')
        c.execute('''CREATE TABLE IF NOT EXISTS race
                     (horse_id int, R int, head_count int, wakuban int, umaban int, popularity int, race_order int, 
                      burden_weight int, odds float, time_gap float, up float, weather text, baba text,
                      raceid int, place text, pace0 float, pace1 float, race_type text, distance int, 
                      weight int, weight_change int, date datetime, time datetime)''')
        conn.commit()
if True:
    insert_sql = 'insert into race values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
    comm_datas = []
    with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
        c = conn.cursor()
        timer = Timer()
        with open('./error_datalog.txt', 'w') as errorlog:
            for i, url in enumerate(tqdm(sorted(list(set(urls)))[::-1])):
                if i <= 3602:
                    continue

                res = requests.get(url)
                time.sleep(0.2)
                soup = BeautifulSoup(res.content, 'html.parser')
                try:
                    all_rows = soup.find('table', class_='db_h_race_results nk_tb_common').find_all('tr')
                except AttributeError:
                    errorlog.write(str((url, i)))
                for row, tr in enumerate(all_rows):
                    if row == 0:
                        continue
                    tds = tr.find_all("td")
                    try:
                        data = tds2data(tds, url)
                    except (ValueError, IndexError, TypeError, AttributeError, KeyError) as e:
                        errorlog.write(str((url, i, row)))
                    if len(data) != 23:
                        continue
                    comm_datas.append(data)
                if i % 50 == 2:
                    c.executemany(insert_sql, comm_datas)
                    conn.commit()
                    comm_datas = []
                    timer.time(f'end {i}')
        c.executemany(insert_sql, comm_datas)
        conn.commit()
        timer.time('end')

/Users/ht/.pyenv/versions/miniconda3-latest/envs/cert/lib/python3.7/site-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


end 3652
abs_time: 88.578684091568
rap_time: 88.578684091568


In [144]:
tds_bag = tds

In [156]:
url

'https://db.netkeiba.com/horse/2013100111/'

In [157]:
res = requests.get(urls[3])
time.sleep(0.2)
soup = BeautifulSoup(res.content, 'html.parser')
all_rows = soup.find('table', class_='db_h_race_results nk_tb_common').find_all('tr')
for row, tr in enumerate(all_rows[1:]):
    tds = tr.find_all("td")
    break
tds

[<td><a href="/race/list/20191215/">2019/12/15</a></td>,
 <td><a href="/race/sum/06/20191215/">5中山6</a></td>,
 <td>晴</td>,
 <td class="txt_right">7</td>,
 <td class=""><a href="/race/201906050607/" title="3歳以上1勝クラス">3歳以上1勝クラス</a></td>,
 <td>
 <a href="/race/movie/201906050607" target="_blank" title="3歳以上1勝クラスの映像"><img border="0" src="/style/netkeiba.ja/image/icon_douga.png"/></a>
 </td>,
 <td class="txt_right">14</td>,
 <td class="txt_right">6</td>,
 <td class="txt_right">9</td>,
 <td class="txt_right">36.2</td>,
 <td class="txt_right">9</td>,
 <td class="bml txt_right">4</td>,
 <td>
 <a href="/jockey/01127/" title="丸山元気">丸山元気</a>
 </td>,
 <td>57</td>,
 <td>ダ2500</td>,
 <td>良</td>,
 <td class="bml txt_right">
 <div align="center">
 <a data-theme="07002" href="https://regist.netkeiba.com/?pid=premium?service=p44" id="a_monthly_goods_link_01">**</a>
 </div>
 </td>,
 <td class="txt_right">2:44.3</td>,
 <td class="txt_right">1.5</td>,
 <td class="bml txt_right">
 <div style="text-align:cen

In [173]:
np.array(tds).shape

(28,)

In [170]:
len(tds)

28

In [175]:
tds_bag[0][0]

KeyError: 0

In [174]:
np.array(tds_bag)

KeyError: 0

In [97]:
res = requests.get(urls[0])
time.sleep(0.2)
soup = BeautifulSoup(res.content, 'html.parser')
tds

KeyError: 0

In [137]:
list(tds)[[1,2,3]]

TypeError: list indices must be integers or slices, not list

In [139]:
for i in list(tds):
    print(1)
    print(i)
    print(type(i))

1
<td><a href="/race/list/20190202/">2019/02/02</a></td>
<class 'bs4.element.Tag'>
1
<td><a href="/race/sum/B6/20190202/">オースト</a></td>
<class 'bs4.element.Tag'>
1
<td> </td>
<class 'bs4.element.Tag'>
1
<td class="txt_right">7</td>
<class 'bs4.element.Tag'>
1
<td class=""><a href="/race/2019B6a00107/" title="ハンデ戦">ハンデ戦</a></td>
<class 'bs4.element.Tag'>
1
<td>

 
</td>
<class 'bs4.element.Tag'>
1
<td class="txt_right">10</td>
<class 'bs4.element.Tag'>
1
<td class="txt_right"> </td>
<class 'bs4.element.Tag'>
1
<td class="txt_right">4</td>
<class 'bs4.element.Tag'>
1
<td class="txt_right">31.0</td>
<class 'bs4.element.Tag'>
1
<td class="txt_right">7</td>
<class 'bs4.element.Tag'>
1
<td class="rank_3 txt_right">3</td>
<class 'bs4.element.Tag'>
1
<td>

J.Co
</td>
<class 'bs4.element.Tag'>
1
<td>57.5</td>
<class 'bs4.element.Tag'>
1
<td>芝1500</td>
<class 'bs4.element.Tag'>
1
<td>重</td>
<class 'bs4.element.Tag'>
1
<td class="bml txt_right">


 
</td>
<class 'bs4.element.Tag'>
1
<td class="tx

In [140]:
np.array([str(td) for td in tds])

array(['<td><a href="/race/list/20190202/">2019/02/02</a></td>',
       '<td><a href="/race/sum/B6/20190202/">オースト</a></td>', '<td> </td>',
       '<td class="txt_right">7</td>',
       '<td class=""><a href="/race/2019B6a00107/" title="ハンデ戦">ハンデ戦</a></td>',
       '<td>\n\n\xa0\n</td>', '<td class="txt_right">10</td>',
       '<td class="txt_right">\xa0</td>', '<td class="txt_right">4</td>',
       '<td class="txt_right">31.0</td>', '<td class="txt_right">7</td>',
       '<td class="rank_3 txt_right">3</td>', '<td>\n\nJ.Co\n</td>',
       '<td>57.5</td>', '<td>芝1500</td>', '<td>重</td>',
       '<td class="bml txt_right">\n\n\n\xa0\n</td>',
       '<td class="txt_right">\xa0</td>',
       '<td class="txt_right">\xa0</td>',
       '<td class="bml txt_right">\n\n\n\xa0\n</td>', '<td>5-5</td>',
       '<td>\xa0</td>', '<td class="bml">\xa0</td>', '<td>計不</td>',
       '<td align="center" class="bml" nowrap="">\n\n\xa0\n</td>',
       '<td class="bml" nowrap="">\n\n\n\xa0\n</td>', '<td>\xa

In [94]:
res = requests.get(urls[2482])
time.sleep(0.2)
soup = BeautifulSoup(res.content, 'html.parser')
all_rows = soup.find('table', class_='db_h_race_results nk_tb_common').find_all('tr')

## データセットの作成
データベースから各レースごとに最大24馬の過去五レースをとってくる
ない分は0埋め

## モデル作成
(A)過去5レース + そのレースの情報　からスコアづけするような部分 + (B)出走馬数ぶんのスコアから1位予想する部分　+ (C)1位確率とoddsから良い買い方を選択する部分

((A)*24)+(B)でまず学習、その後
(C)を学習

考え中

- とりあえず全部全結合で、Bは一層? Cも1層？
- A, B は予測にodds は用いない
- Aはlstmと合わせる?

### A
入力: 

- 当日情報
    - 数値データ
        - 前のレースから何日
        - 頭数
        - 枠版
        - 馬番
        - 斤量
        - 重量
        - 重量変化
        - 距離
        - R 
    - カテゴリ(one-hot でok)
        - レース種類
        - 天気
        - 馬場
- 過去五レース(当日情報で扱うデータに加え)
    - 数値データ
        - 着順
        - odds
        - タイム
        - 着差
        - ペース
        - 上り
    - カテゴリ(one-hot でok)
    
出力: score(float)

### B
入力: score*24

出力: 1位確率*24

loss = cross entoropy AとBで一回学習させる

### C
入力: 1位確率*24, odds*24

出力: 買うべき金額

loss = 金額の増減 でcのみ学習？
